In [1]:
# 约瑟夫环问题
# 假设有n个人， 从第一个人开始报数， 数到m的人被淘汰
# person = ['Bob', 'Jack', 'Allen', 'Tony', 'Peter', 'Rose', 'Wade']


# 约瑟夫环是一个容器
# 容器一般具有的功能有 增 删 查 改
import copy
import csv
import zipfile

class Person():
    def __init__(self, name = '', age = -1):
        self.name = name
        self.age = age

    @classmethod
    def create_from_txt_or_csv_reader(cls, row):
        some_oen = cls()
        row = row.strip()
        some_oen.name, some_oen.age = item.split(',')[0], int(item.split(',')[1])
        return some_oen

class JosephusRing:
    def __init__(self, reader = None):
        self.people = []
        self.start = -1
        self.step = -1
        if reader:
            for row in reader:
                self.people.append(Person.create_from_txt_or_csv_reader(row))

    # 增功能
    def append(self, obj):
        self.people.append(obj)

    # 删功能
    def pop(self, index):
        self.people.pop(index)
    
    # 查功能
    # 查找整个容器内容
    def query_list(self):
        temp = copy.copy(self.people)
        size = len(temp)
        if size == 0:
            return None
        
        index = self.start - 1
        ret = []
        for i in range(size):
            index = (index + self.step - 1) % len(temp)
            ret.append(temp.pop(index))
        return ret

    # 用生成器来进行一个一个输出 利用__next__
    def iter(self):
        temp = self.people
        start_index = self.start - 1
        while temp:    
            index = (start_index + self.step - 1) % len(temp)
            ret = temp.pop(index)
            start_index = index
            yield ret
        
        return None

    @classmethod
    def create_from_txt(cls, path):
        if '.txt' not in path:
            raise Exception('wrong path')
        
        obj = cls()
        with open(path) as txt_file:
            for row in txt_file:
                row = row.strip()
                temp = row.split(',')
                name = temp[0]
                age = int(temp[1])
                obj.append(Person(name, age))
        return obj
    
    @classmethod
    def create_from_csv(cls, path):
        if '.csv' not in path:
            raise Exception('wrong path')  

        obj = cls()
        with open(path) as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                name = row[0]
                age = row[1]
                obj.append(Person(name, age))
        return obj

    @classmethod
    def create_from_zip(cls, path, target_file):
        if not zipfile.is_zipfile(path):
            raise Exception('wrong path')
        
        obj = cls()
        with zipfile.ZipFile(path) as z:
            open_file = z.extract(target_file)
            
            if '.txt' in target_file:
                obj = JosephusRing.create_from_txt(open_file)
            
            if '.csv' in target_file:
                obj = JosephusRing.create_from_csv(open_file)

            return obj




In [8]:
with open('person.txt') as reader:
    jos = JosephusRing(reader)
    for each in jos.people:
        print(each.name, each.age)
print('-------------')
jos.start = 3
jos.step = 4
res = jos.query_list()
for each in res:
    print(each.name, each.age)

Bob 15
Jack 12
Allen 17
Tony 20
Peter 15
Rose 16
Wade 19
-------------
Rose 16
Allen 17
Bob 15
Wade 19
Jack 12
Peter 15
Tony 20
